In [1]:
import sys
import os
from pathlib import Path
# This appends the directory one level up (the root of your project) to the sys.path.
# Modify the path depending on the location of modules you want to import.
sys.path.append(os.path.abspath('../'))

In [2]:
from config.config_managers import DashboardConfigManager
from dataManager import DataManager



2024-11-24 17:58:25 - INFO - PyTorch version 2.2.2 available.


In [3]:
CONFIG_PATH = Path("/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/dashboard-config.yaml")
config_manager = DashboardConfigManager(CONFIG_PATH)
dev_config = config_manager.development_config    

In [4]:

from dash import Dash, dcc, html, Output, Input, State
app = Dash(__name__, suppress_callback_exceptions=True)

app_config = config_manager.app_config
server = app.server  # Flask server instance for caching
variants_data = None

data_manager = DataManager(config_manager, server)

In [5]:
data_manager.load_data()

2024-11-24 17:58:27 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/My Drive
2024-11-24 17:58:27 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2024-11-24 17:58:27 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02


  0%|          | 0/12 [00:00<?, ?it/s]

2024-11-24 17:58:32 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/My Drive
2024-11-24 17:58:32 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2024-11-24 17:58:32 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/conll2003_bert


  0%|          | 0/12 [00:00<?, ?it/s]

{'ANERCorp_CamelLab_arabertv02': DashboardData(analysis_data=       Sentence Ids  Token Positions     Words    Tokens Word Pieces  \
 0                 0                0     [CLS]     [CLS]       [CLS]   
 1                 0                1  الصالحية  الصالحية    الصالحية   
 2                 0                2    المفرق    المفرق      المفرق   
 3                 0                3         -         -           -   
 4                 0                4       غيث       غيث         غيث   
 ...             ...              ...       ...       ...         ...   
 29706           960               10    للوليد     ##ليد  للو, ##ليد   
 29707           960               11        بن        بن          بن   
 29708           960               12      طلال      طلال        طلال   
 29709           960               13         .         .           .   
 29710           960               14     [SEP]     [SEP]       [SEP]   
 
       Core Tokens True Labels Token Selector Id Pred Labels  

In [6]:
analysis_data = data_manager.variants_data['ANERCorp_CamelLab_arabertv02'].analysis_data
errors = analysis_data[analysis_data['Error Type'] != 'No Errors']

In [ ]:
import pandas as pd
from collections import defaultdict, Counter
from seqeval.scheme import auto_detect, Entities
from seqeval.metrics.sequence_labeling import get_entities

class EntityAnnotator:
    def __init__(self, y_true, y_pred):
        """Initialize the annotator with true and predicted labels."""
        self.y_true = y_true
        self.y_pred = y_pred
        self.scheme = auto_detect(self.y_true, False)
    
    def extract_entities(self):
        """Extract entities based on the scheme detected."""
        self.entities_strict_true = Entities(self.y_true, self.scheme, False)
        self.entities_strict_pred = Entities(self.y_pred, self.scheme, False)
        self.entities_true = get_entities(self.y_true)
        self.entities_pred = get_entities(self.y_pred)

    def process_strict_entities(self, y_true, entities_true, sen_id):
        """Process entities strictly, labeling full spans in a sentence."""
        max_len = len(y_true[sen_id])
        results = ['O'] * max_len
        for idx in range(max_len):
            for entity in entities_true[sen_id]:
                _, t, s, e = entity.to_tuple()
                if s == idx and (e-s) > 0:
                    for i in range(e-s):
                        results[s + i] = t
                elif (e-s) == 0:
                    results[s] = t
        return results
    
    def process_non_strict_entities(self, y_true, sen_id):
        """Process entities non-strictly, marking only the start and end of each entity."""
        max_len = len(y_true[sen_id])
        results = ['O'] * max_len
        for entity in get_entities(y_true[sen_id]):
            t, s, e = entity
            if s == e:
                # If start and end are the same, only mark the start
                results[s] = t
            else:
                # Mark all indices from start to end inclusive
                for i in range(s, e + 1):
                    results[i] = t
        return results

    def process_sentences(self, analysis_data, y_data, entities, label_column, strict=False):
        """Annotate sentences with entity information, either strictly or non-strictly."""
        entity_annotations = []
        for sentence_id, sentence_df in analysis_data.groupby('Sentence Ids'):
            if strict: 
                results = self.process_strict_entities(y_data, entities, sentence_id)
            else:
                results = self.process_non_strict_entities(y_data, sentence_id)
            original_series = sentence_df[label_column]
            is_metadata = original_series.apply(lambda x: x not in ['[CLS]', '[SEP]', 'IGNORED'])
            new_series = original_series.copy()
            new_series.loc[is_metadata] = results
            entity_annotations.append(new_series)
        return pd.concat(entity_annotations)

    def annotate_entity_info(self, analysis_data):
        """Add annotated entity information to the analysis data for both true and predicted labels."""
        self.extract_entities()  # Ensure entities are extracted before processing
        analysis_data['Strict True Entities'] = self.process_sentences(analysis_data, self.y_true, self.entities_strict_true.entities, 'True Labels', True)
        analysis_data['Strict Pred Entities'] = self.process_sentences(analysis_data, self.y_pred, self.entities_strict_pred.entities, 'Pred Labels', True)
        analysis_data['True Entities'] = self.process_sentences(analysis_data, self.y_true, self.entities_true, 'True Labels')
        analysis_data['Pred Entities'] = self.process_sentences(analysis_data, self.y_pred, self.entities_pred, 'Pred Labels')
        
        return analysis_data


In [292]:
import json
from collections import defaultdict, Counter
from seqeval.scheme import auto_detect
from seqeval.metrics.sequence_labeling import get_entities
from seqeval.scheme import Entities

file_name = '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02/fine_tuning/evaluation_metrics.json'
with open(file_name, 'r') as file:
    entity_outputs = json.load(file)  # Use json.load() to read file, not json.loads()

def extract_entities(y_data, scheme):
        # Replace with the Entities() logic if provided
        return Entities(y_data, scheme, False)
core_data = analysis_data[analysis_data['Labels']!= -100].copy()


# y_true = entity_outputs['entity_outputs']['y_true']
y_true = core_data.groupby('Sentence Ids')['True Labels'].apply(list).tolist()
# y_pred = entity_outputs['entity_outputs']['y_pred']
y_pred = core_data.groupby('Sentence Ids')['Pred Labels'].apply(list).tolist()


In [293]:
annotator = EntityAnnotator(y_true, y_pred)
updated_analysis_data = annotator.annotate_entity_info(analysis_data)


In [294]:
updated_analysis_data[['True Entities', 'TR Entity']]

,True Entities,TR Entity
0,[CLS],[CLS]
1,LOC,LOC
2,LOC,LOC
3,O,O
4,PERS,PERS
...,...,...
29706,IGNORED,IGNORED
29707,PERS,PERS
29708,PERS,PERS
29709,O,O


In [295]:
(updated_analysis_data['True Entities'] != updated_analysis_data['TR Entity']).value_counts()

False    29711
Name: count, dtype: int64

In [296]:
updated_analysis_data[updated_analysis_data['True Entities'] != updated_analysis_data['Strict True Entities']][['Sentence Ids', 'True Entities', 'TR Entity']]

,Sentence Ids,True Entities,TR Entity
379,10,PERS,PERS
415,11,ORG,ORG
1171,32,PERS,PERS
1196,35,MISC,MISC
1197,35,MISC,MISC
...,...,...,...
28947,941,PERS,PERS
28948,941,PERS,PERS
29068,946,PERS,PERS
29539,957,PERS,PERS


In [297]:
sen_id = 379
get_entities(y_true[sen_id])

[('LOC', 13, 13), ('MISC', 17, 19), ('MISC', 21, 21), ('PERS', 28, 29)]

In [298]:
ts = updated_analysis_data[updated_analysis_data['Sentence Ids'] == sen_id]

ts[~ts['True Labels'].isin(['[CLS]', '[SEP]', 'IGNORED'])][['Sentence Ids', 'True Entities', 'Strict True Entities', 'TR Entity', 'True Labels']].reset_index()

,index,Sentence Ids,True Entities,Strict True Entities,TR Entity,True Labels
0,11860,379,O,O,O,O
1,11862,379,O,O,O,O
2,11863,379,O,O,O,O
3,11864,379,O,O,O,O
4,11865,379,O,O,O,O
5,11866,379,O,O,O,O
6,11867,379,O,O,O,O
7,11869,379,O,O,O,O
8,11870,379,O,O,O,O
9,11873,379,O,O,O,O


In [ ]:
scheme = auto_detect(y_true, False)
entities_true = extract_entities(y_true)
entities_pred = extract_entities(y_pred)
true_entities = get_entities(y_true)

In [159]:
def process_strict_entities(y_true, entities_true, sen_id):
    """
    Process entities for a given sentence ID and return a list of results where entities are labeled,
    and non-entity indices are marked as 'O'.

    Args:
    y_true (dict): Dictionary where keys are sentence IDs and values are lists of true labels.
    entities_true (dict): Dictionary where keys are sentence IDs and values are lists of entity objects with a to_tuple method.
    sen_id (int): Sentence ID to process.

    Returns:
    list: A list of labels with entities labeled accordingly and other indices as 'O'.
    """
    # Determine the length needed for the results list
    max_len = len(y_true[sen_id])

    # Initialize results with 'O' for all expected indices
    results = ['O'] * max_len

    # Iterate over each index in the range of y_true for the given sentence ID
    for idx in range(max_len):
        # Check for entities at this index
        for entity in entities_true[sen_id]:
            _, t, s, e = entity.to_tuple()
            # Check if the entity starts at this index and has length
            if s == idx and (e-s) > 0:
                for i in range(e-s):
                    results[s + i] = t  # Replace 'some_label' with your intended label
            elif (e-s) == 0:
                results[s] = t  # Set your intended label for zero-length entities
            # If none of the conditions are met, it will retain 'O'

    return results


In [160]:
analysis_data.columns

Index(['Sentence Ids', 'Token Positions', 'Words', 'Tokens', 'Word Pieces',
       'Core Tokens', 'True Labels', 'Token Selector Id', 'Pred Labels',
       'Agreements', 'X', 'Y', 'Labels', 'Losses', 'Token Ids', 'Global Id',
       'True Silhouette Score', 'Pred Silhouette Score', 'K=3',
       'Boundary Clusters', 'K=4', 'Entity Clusters', 'K=9', 'Token Clusters',
       'Consistency Count', 'Inconsistency Count', 'Total Train Occurrences',
       'Local Token Entropy', 'Token Max Entropy', 'Dataset Token Entropy',
       'Local Word Entropy', 'Word Max Entropy', 'Dataset Word Entropy',
       'Tokenization Rate', 'TR Entity', 'PR Entity', 'Error Type',
       'O Confidence', 'B-PERS Confidence', 'I-PERS Confidence',
       'B-ORG Confidence', 'I-ORG Confidence', 'B-LOC Confidence',
       'I-LOC Confidence', 'B-MISC Confidence', 'I-MISC Confidence',
       'Prediction Entropy', 'Prediction Max Entropy', 'Token Confidence',
       'Variability', 'Pre X', 'Pre Y', 'Consistency Ratio',

In [169]:
entity_annotations = []
for sentence_id, sentence_df in analysis_data.groupby('Sentence Ids'):
    results = process_strict_entities(y_true, entities_true.entities, sentence_id)
    original_series = sentence_df['True Labels']
    is_metadata = original_series.apply(lambda x: x not in ['[CLS]', '[SEP]', 'IGNORED'])
    new_series = original_series.copy()
    new_series.loc[is_metadata] = results
    entity_annotations.append(new_series)


In [170]:
pd.concat(entity_annotations)

0          [CLS]
1            LOC
2            LOC
3              O
4           PERS
          ...   
29706    IGNORED
29707       PERS
29708       PERS
29709          O
29710      [SEP]
Name: True Labels, Length: 29711, dtype: object

In [250]:
sen_id = 18
data = analysis_data[~analysis_data['TR Entity'].isin(['[CLS]', '[SEP]', 'IGNORED'])]
sen_data = data[data['Sentence Ids'] == sen_id].reset_index()
sen_data[['Sentence Ids', 'True Labels', 'TR Entity']]

,Sentence Ids,True Labels,TR Entity
0,18,O,O
1,18,O,O
2,18,O,O
3,18,B-LOC,LOC
4,18,I-LOC,LOC
5,18,O,O
6,18,O,O
7,18,O,O
8,18,O,O
9,18,B-ORG,ORG


In [251]:
lresults

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [252]:
# Determine the length needed for the results list
max_len = len(y_true[sen_id])

# Initialize results with 'O' for all expected indices
results = ['O'] * max_len

# Iterate over each index in the range of y_true[694]
for idx in range(max_len):
    # Check for entities at this index
    for entity in entities_true.entities[sen_id]:
        sen_id, t, s, e = entity.to_tuple()
        # Check if the entity starts at this index and has length
        if s == idx and (e-s) > 0:
            for i in range(e-s):
                results[s + i] = t  # Replace 'some_label' with your intended label
        elif (e-s) == 0:
            results[s] = t  # Again, set your intended label for zero-length entities
        # If none of the conditions are met, it will retain 'O'


In [259]:
# Determine the length needed for the results list
max_len = len(y_true[sen_id])

# Initialize results with 'O' for all expected indices
lresults = ['O'] * max_len

# Iterate over each index in the range of y_true[694]
for idx in range(max_len):
    # Check for entities at this index
    for entity in get_entities(y_true[sen_id]):
        t, s, e = entity
        # Check if the entity starts at this index and has length
        
        if s == e:
            # If start and end are the same, only mark the start
            lresults[s] = t
        else:
            # Mark all indices from start to end inclusive
            for i in range(s, e + 1):
                lresults[i] = t
        


In [260]:
lresults

['O',
 'O',
 'O',
 'LOC',
 'LOC',
 'O',
 'O',
 'O',
 'O',
 'ORG',
 'O',
 'PERS',
 'PERS',
 'PERS',
 'O',
 'O',
 'O',
 'O',
 'O',
 'LOC',
 'O']

In [142]:
original_series = analysis_data[analysis_data["Sentence Ids"] == sen_id]['True Labels']
is_metadata = original_series.apply(lambda x: x not in ['[CLS]', '[SEP]', 'IGNORED'])


In [ ]:
new_series = original_series.copy()
non_metadata_indices = original_series.index[is_metadata]
new_series.loc[non_metadata_indices] = results


In [152]:
original_series

21067    [CLS]
21068    I-ORG
21069    B-ORG
21070    I-ORG
21071        O
         ...  
21127        O
21128        O
21129        O
21130        O
21131    [SEP]
Name: True Labels, Length: 65, dtype: object

In [154]:
sen_data['TR Entity']

0     ORG
1     ORG
2     ORG
3       O
4       O
5       O
6       O
7       O
8     ORG
9     ORG
10      O
11      O
12      O
13      O
14      O
15      O
16      O
17      O
18      O
19      O
20      O
21      O
22      O
23      O
24      O
25      O
26    ORG
27      O
28    ORG
29      O
30      O
31      O
32      O
33      O
34      O
35    ORG
36    ORG
37      O
38      O
39      O
40      O
41      O
42    ORG
43      O
44      O
45      O
46    ORG
47      O
48      O
49      O
50      O
51      O
52      O
53      O
54      O
55      O
Name: TR Entity, dtype: object

In [155]:
new_series

21067    [CLS]
21068        O
21069      ORG
21070      ORG
21071        O
         ...  
21127        O
21128        O
21129        O
21130        O
21131    [SEP]
Name: True Labels, Length: 65, dtype: object

In [89]:
get_entities(y_true[sen_id])

[('PERS', 6, 7), ('MISC', 14, 15), ('MISC', 17, 17), ('LOC', 39, 39)]

In [90]:
entities_true.entities[sen_id]


[(163, PERS, 6, 8), (163, MISC, 14, 16), (163, LOC, 39, 40)]

In [100]:
for i, e in enumerate(lresults
):
    print(i, e)

0 O
1 O
2 O
3 O
4 O
5 O
6 PERS
7 PERS
8 O
9 O
10 O
11 O
12 O
13 O
14 MISC
15 MISC
16 O
17 MISC
18 O
19 O
20 O
21 O
22 O
23 O
24 O
25 O
26 O
27 O
28 O
29 O
30 O
31 O
32 O
33 O
34 O
35 O
36 O
37 O
38 O
39 LOC
40 O


In [ ]:
# Determine the length needed for the results list
max_len = len(y_true[0])

# Initialize results with 'O' for all expected indices
results = ['O'] * max_len

# Iterate over each index in the range of y_true[694]
for idx in range(max_len):
    # Check for entities at this index
    for entity in entities_true.entities[0]:
        sen_id, t, s, e = entity.to_tuple()
        # Check if the entity starts at this index and has length
        if s == idx and (e-s) > 0:
            for i in range(e-s):
                results[s + i] = t  # Replace 'some_label' with your intended label
        elif (e-s) == 0:
            results[s] = t  # Again, set your intended label for zero-length entities
        # If none of the conditions are met, it will retain 'O'


In [76]:
results

['O',
 'ORG',
 'ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'ORG',
 'ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'ORG',
 'O',
 'ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'ORG',
 'ORG',
 'O',
 'O',
 'O',
 'O',
 'O',
 'ORG']

In [7]:
errors[errors['Error Type'] == 'Type'][['True Labels', 'Pred Labels', 'Error Type']].sample(60)

,True Labels,Pred Labels,Error Type
6020,B-MISC,B-ORG,Type
4493,B-MISC,B-LOC,Type
6184,B-PERS,B-LOC,Type
23599,I-ORG,I-MISC,Type
14521,B-LOC,B-MISC,Type
8241,B-ORG,B-LOC,Type
6215,B-PERS,B-ORG,Type
6135,B-PERS,B-ORG,Type
7966,I-MISC,I-LOC,Type
19374,I-MISC,I-LOC,Type


In [9]:
errors[['True Labels', 'Pred Labels', 'Error Type']].sample(60)

,True Labels,Pred Labels,Error Type
8241,B-ORG,B-LOC,Type
29003,B-ORG,O,Exclusion
25590,I-MISC,O,Exclusion
12736,B-LOC,O,Exclusion
10272,I-PERS,O,Exclusion
21751,B-ORG,B-LOC,Type
27543,B-PERS,I-PERS,Chunk
1188,I-MISC,O,Exclusion
7160,B-PERS,B-ORG,Type
25330,O,B-PERS,Chunk


In [ ]:
def annotate_error(true_label, pred_label):
        # If both are the same, it's correct (no error)
        if true_label == pred_label:
            return "No Errors"
        
        # Handle cases where one or both labels are 'O'
        if true_label == 'O' and pred_label != 'O':
            return "Chunk"  # False entity predicted
        if true_label != 'O' and pred_label == 'O':
            return "Exclusion"  # Missed entity and chunk boundary
        
        # Extract entity types without position tags (like "B-", "I-")
        true_entity = true_label.split("-")[-1] if "-" in true_label else true_label
        pred_entity = pred_label.split("-")[-1] if "-" in pred_label else pred_label

        # If entity types are different (e.g., LOC vs. PER)
        if true_entity != pred_entity:
            # If both entity type and position (B- vs I-) are wrong
            return "Type and Chunk" if true_label[0] != pred_label[0] else "Type"

        # If entity types are the same but position tags (B- vs I-) are wrong
        return "Chunk"


In [136]:
import json

file_name = '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02/fine_tuning/evaluation_metrics.json'
with open(file_name, 'r') as file:
    entity_outputs = json.load(file)  # Use json.load() to read file, not json.loads()


In [137]:
y_true = entity_outputs['entity_outputs']['y_true']
y_pred = entity_outputs['entity_outputs']['y_pred']

In [138]:
from seqeval.metrics import f1_score, classification_report, f1_score
from seqeval.scheme import IOB2

print(classification_report(y_true, y_pred, mode='strict', digits=4))

f1_score(y_true, y_pred, mode='strict', scheme=IOB2,  average='micro')


              precision    recall  f1-score   support

         LOC     0.8927    0.9341    0.9129       668
        MISC     0.7720    0.6340    0.6963       235
         ORG     0.7842    0.7511    0.7673       450
        PERS     0.8599    0.8438    0.8518       858

   micro avg     0.8476    0.8299    0.8387      2211
   macro avg     0.8272    0.7908    0.8071      2211
weighted avg     0.8451    0.8299    0.8365      2211



0.8386654478976234

In [139]:
from seqeval.metrics import f1_score, classification_report

print(classification_report(y_true, y_pred, mode=None, digits=4))


              precision    recall  f1-score   support

         LOC     0.8919    0.9275    0.9094       676
        MISC     0.7366    0.6214    0.6741       243
         ORG     0.7630    0.7364    0.7494       459
        PERS     0.8835    0.8298    0.8558       905

   micro avg     0.8483    0.8178    0.8327      2283
   macro avg     0.8187    0.7788    0.7972      2283
weighted avg     0.8461    0.8178    0.8310      2283



In [140]:
from seqeval.metrics import f1_score

# Ground truth (true labels)
y_true = [
    ['O', 'B-PER', 'I-PER', 'O', 'B-LOC', 'O'],
    ['O', 'B-ORG', 'I-ORG', 'O']
]

# Predicted labels (with minor errors)
y_pred = [
    ['O', 'B-PER', 'O', 'O', 'B-LOC', 'O'],  # Misses I-PER
    ['O', 'B-ORG', 'O', 'O']                # Misses I-ORG
]

# Default mode
f1_default = f1_score(y_true, y_pred, average='micro', mode=None)

# Strict mode
f1_strict = f1_score(y_true, y_pred, average='micro', mode='strict')

print("Default Mode F1 Score:", f1_default)
print("Strict Mode F1 Score:", f1_strict)


Default Mode F1 Score: 0.3333333333333333
Strict Mode F1 Score: 0.3333333333333333


In [141]:
from seqeval.metrics import f1_score, classification_report
from seqeval.scheme import IOB1
# Define data
y_true = [['B-PER', 'I-PER', 'O', 'B-ORG', 'I-ORG', 'O']]
y_pred = [['B-PER', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'O']]

# Strict mode, no scheme
# f1_no_scheme_strict = f1_score(y_true, y_pred, average='micro', mode='strict', scheme=None)
f1_no_scheme_strict = classification_report(y_true, y_pred, mode='strict', zero_division='Warn')
no_strict = classification_report(y_true, y_pred, mode=None,)

print("Strict Mode with No Scheme F1 Score:", f1_no_scheme_strict)

print("No Strict Mode with F1 Score:", no_strict)




Strict Mode with No Scheme F1 Score:               precision    recall  f1-score   support

         ORG       1.00      1.00      1.00         1
         PER       0.00      0.00      0.00         1

   micro avg       0.50      0.50      0.50         2
   macro avg       0.50      0.50      0.50         2
weighted avg       0.50      0.50      0.50         2

No Strict Mode with F1 Score:               precision    recall  f1-score   support

         ORG       0.50      1.00      0.67         1
         PER       0.00      0.00      0.00         1

   micro avg       0.33      0.50      0.40         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



In [142]:

entity_y_true = get_entities(y_true)
entity_y_pred = get_entities(y_pred)

# Example usage
# conf_matrix = calculate_confusion_matrix([e.to_tuple()[1:] for sen in entities_true.entities for e in sen], [e.to_tuple()[1:] for sen in entities_pred.entities for e in sen])



conf_matrix = calculate_confusion_matrix(entity_y_true, entity_y_pred)
print(conf_matrix)
fn_errors = compute_false_negatives(entity_y_true, entity_y_pred)
fp_errors = compute_false_positives(entity_y_true, entity_y_pred)

print("False Negatives:", dict(fn_errors))
print("False Positives:", dict(fp_errors))
1

{'ORG': {'TP': 1, 'FP': 1, 'FN': 0}, 'PER': {'TP': 0, 'FP': 1, 'FN': 1}}


KeyError: (0, 0)

In [ ]:
entity_y_true

[('PER', 0, 1), ('ORG', 3, 4)]

In [ ]:
entity_y_pred

[('PER', 0, 0), ('ORG', 1, 1), ('ORG', 3, 4)]

In [ ]:


entities_true = Entities(y_true, scheme, False)
entities_pred = Entities(y_pred, scheme, False)
true_entity_type = flatten_strict_entities(entities_true)
pred_entity_type = flatten_strict_entities(entities_pred)
# Example usage
# conf_matrix = calculate_confusion_matrix([e.to_tuple()[1:] for sen in entities_true.entities for e in sen], [e.to_tuple()[1:] for sen in entities_pred.entities for e in sen])



conf_matrix = calculate_confusion_matrix(true_entity_type, pred_entity_type)
print(conf_matrix)
fn_errors = compute_false_negatives(true_entity_type, pred_entity_type)
fp_errors = compute_false_positives(true_entity_type, pred_entity_type)

print("False Negatives:", dict(fn_errors))
print("False Positives:", dict(fp_errors))
1

{'ORG': {'TP': 1, 'FP': 0, 'FN': 0}, 'PER': {'TP': 0, 'FP': 1, 'FN': 1}}
False Negatives: {'PER': Counter({'Boundary': 1})}
False Positives: {'PER': Counter({'Boundary': 1})}


1

In [ ]:
entities_true.entities

[[(0, PER, 0, 2), (0, ORG, 2, 4)]]

In [ ]:
entities_pred.entities

[[(0, PER, 0, 1), (0, ORG, 2, 4)]]

In [ ]:
print(no_strict)

              precision    recall  f1-score   support

         ORG       0.50      1.00      0.67         1
         PER       0.00      0.00      0.00         1

   micro avg       0.33      0.50      0.40         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



In [ ]:
from seqeval.scheme import IOB2, Tokens, auto_detect


In [ ]:
scheme = auto_detect(y_true, False)

In [ ]:
scheme

seqeval.scheme.IOB2

In [ ]:
tokens = Tokens(['B-PER', 'I-PER', 'O', 'B-LOC'], scheme)

In [ ]:
# outputs = {
#     'y_true': [['O', 'O', 'B-MISC', 'I-MISC', 'B-MISC', 'O', 'O'], ['B-PER', 'I-PER', 'O']],
# 	'y_pred': [['O', 'O', 'B-MISC', 'I-LOC', 'B-MISC', 'I-MISC', 'O'], ['B-PER', 'I-PER', 'O']]

# }


In [144]:
from seqeval.metrics.sequence_labeling import get_entities
# Example usage
entity_y_true = get_entities(entity_outputs['entity_outputs']['y_true'])
entity_y_pred = get_entities(entity_outputs['entity_outputs']['y_pred'])

from seqeval.scheme import Entities
entities_true = Entities(entity_outputs['entity_outputs']['y_true'], scheme, False)
entities_pred = Entities(entity_outputs['entity_outputs']['y_pred'], scheme, False)



In [ ]:
from collections import defaultdict, Counter

def flatten_strict_entities(entities):
    return [e.to_tuple()[1:] for sen in entities.entities for e in sen]

def calculate_confusion_matrix(y_true, y_pred):
    # Initialize confusion matrix data structure
    types = set([ent[0] for ent in y_true]).union([ent[0] for ent in y_pred])
    confusion_matrix = {typ: {'TP': 0, 'FP': 0, 'FN': 0} for typ in types}


    # Track matched predictions to avoid counting them more than once
    matched_pred_indices = set()

    # Check each true entity against predicted entities
    for true_ent in y_true:
        true_type, true_start, true_end = true_ent
        match_found = False

        for idx, pred_ent in enumerate(y_pred):
            pred_type, pred_start, pred_end = pred_ent

            if idx not in matched_pred_indices and true_type == pred_type and true_start == pred_start and true_end == pred_end:
                confusion_matrix[true_type]['TP'] += 1
                matched_pred_indices.add(idx)
                match_found = True
                break
        
        if not match_found:
            confusion_matrix[true_type]['FN'] += 1


    # Any unmatched prediction is a false positive
    for idx, pred_ent in enumerate(y_pred):
        if idx not in matched_pred_indices:
            pred_type = pred_ent[0]
            confusion_matrix[pred_type]['FP'] += 1

    return confusion_matrix

entities_true = Entities(entity_outputs['entity_outputs']['y_true'], scheme, False)
entities_pred = Entities(entity_outputs['entity_outputs']['y_pred'], scheme, False)
true_entity_type = flatten_strict_entities(entities_true)
pred_entity_type = flatten_strict_entities(entities_pred)
# Example usage
# conf_matrix = calculate_confusion_matrix([e.to_tuple()[1:] for sen in entities_true.entities for e in sen], [e.to_tuple()[1:] for sen in entities_pred.entities for e in sen])




{'MISC': {'TP': 151, 'FP': 54, 'FN': 92}, 'PERS': {'TP': 751, 'FP': 99, 'FN': 154}, 'LOC': {'TP': 627, 'FP': 76, 'FN': 49}, 'ORG': {'TP': 338, 'FP': 105, 'FN': 121}}


In [ ]:
conf_matrix = calculate_confusion_matrix(entity_y_true, entity_y_pred)
print(conf_matrix)

{'MISC': {'TP': 151, 'FP': 54, 'FN': 92}, 'PERS': {'TP': 751, 'FP': 99, 'FN': 154}, 'LOC': {'TP': 627, 'FP': 76, 'FN': 49}, 'ORG': {'TP': 338, 'FP': 105, 'FN': 121}}


In [ ]:
(627+151+751+338) / ((627+92)+(151+154)+(751+49)+(338+121))

0.8177836180464302

In [ ]:
1867 / 2201

0.8482507950931395

In [ ]:
conf_matrix = calculate_confusion_matrix(true_entity_type, pred_entity_type)
print(conf_matrix)

{'MISC': {'TP': 166, 'FP': 27, 'FN': 69}, 'PERS': {'TP': 782, 'FP': 60, 'FN': 76}, 'LOC': {'TP': 642, 'FP': 57, 'FN': 26}, 'ORG': {'TP': 386, 'FP': 45, 'FN': 64}}


In [ ]:
total_metrics = {}
for metric in ['TP', 'FP', 'FN']:
    total_metrics[metric] = sum(details[metric] for details in conf_matrix.values())


In [ ]:
total_metrics

{'TP': 1976, 'FP': 189, 'FN': 235}

In [ ]:
1976 / (1976+189)

0.91270207852194

In [152]:
entity_false_positives

{'LOC': Counter({'ORG': 20, 'MISC': 6, 'PERS': 6})}

In [184]:

from collections import defaultdict, Counter

def compute_false_negatives(y_true, y_pred):
    fn_counts = defaultdict(Counter)
    true_indexed = {(t[1], t[2]): t[0] for t in y_true}  # Index true entities by boundaries
    pred_indexed = {(p[1], p[2]): p[0] for p in y_pred}  # Index predicted entities by boundaries

    # Iterate through true entities to find false negatives
    for (t_start, t_end), t_type in true_indexed.items():
        if (t_start, t_end) not in pred_indexed or pred_indexed[(t_start, t_end)] != t_type:
            # No matching prediction or type mismatch at the same position
            matched_type = pred_indexed.get((t_start, t_end), 'Boundary')
            fn_counts[t_type][matched_type] += 1

    return fn_counts

def compute_false_positives(y_true, y_pred):
    fp_counts = defaultdict(Counter)
    true_indexed = {(t[1], t[2]): t[0] for t in y_true}  # Index true entities by boundaries
    pred_indexed = {(p[1], p[2]): p[0] for p in y_pred}  # Index predicted entities by boundaries

    # Iterate through predicted entities to find false positives
    for (p_start, p_end), p_type in pred_indexed.items():
        if (p_start, p_end) not in true_indexed or true_indexed[(p_start, p_end)] != p_type:
            # No matching true entity or type mismatch at the same position
            matched_type = true_indexed.get((p_start, p_end), 'Boundary')
            fp_counts[p_type][matched_type] += 1

    return fp_counts

# Example usage
fn_errors = compute_false_negatives(entity_y_true, entity_y_pred)
fp_errors = compute_false_positives(entity_y_true, entity_y_pred)

print("False Negatives:", dict(fn_errors))
print("False Positives:", dict(fp_errors))
1

False Negatives: {'LOC': Counter({'Boundary': 42, 'MISC': 3, 'ORG': 3, 'PERS': 1}), 'PERS': Counter({'Boundary': 130, 'ORG': 13, 'LOC': 6, 'MISC': 5}), 'ORG': Counter({'Boundary': 84, 'LOC': 20, 'PERS': 11, 'MISC': 6}), 'MISC': Counter({'Boundary': 77, 'ORG': 9, 'LOC': 6})}
False Positives: {'LOC': Counter({'Boundary': 44, 'ORG': 20, 'MISC': 6, 'PERS': 6}), 'MISC': Counter({'Boundary': 40, 'ORG': 6, 'PERS': 5, 'LOC': 3}), 'ORG': Counter({'Boundary': 80, 'PERS': 13, 'MISC': 9, 'LOC': 3}), 'PERS': Counter({'Boundary': 87, 'ORG': 11, 'LOC': 1})}


1

In [312]:
import plotly.express as px
import pandas as pd

# Your original data
data = conf_matrix

# Prepare lists for DataFrame construction
actual = []
predicted = []
counts = []

for (act, pred), count in data.items():
    actual.append(act)
    predicted.append('None' if pred is None else pred)  # Replace None with 'None' for better visualization
    counts.append(count)

# Create DataFrame
df = pd.DataFrame({'Actual': actual, 'Predicted': predicted, 'Count': counts})

# Pivot to format suitable for heatmap
pivot_table = df.pivot(index='Actual', columns='Predicted', values='Count').fillna(0)

# Generate heatmap
fig = px.imshow(pivot_table,
                labels=dict(x="Predicted Entity Type", y="Actual Entity Type", color="Count"),
                x=pivot_table.columns,
                y=pivot_table.index,
                text_auto=True,
                aspect="auto")

fig.update_layout(
    title="Entity Recognition Confusion Matrix",
    xaxis_title="Predicted Entity Type",
    yaxis_title="Actual Entity Type"
)

fig.show()


In [331]:
errors

defaultdict(<function __main__.calculate_confusion_matrix.<locals>.<lambda>()>,
            {'PERS': defaultdict(int,
                         {'MISC': 10, 'ORG': 26, 'LOC': 12, 'Unmatched': 99}),
             'LOC': defaultdict(int,
                         {'PERS': 2, 'MISC': 6, 'ORG': 6, 'Unmatched': 76}),
             'MISC': defaultdict(int, {'LOC': 12, 'ORG': 18, 'Unmatched': 54}),
             'ORG': defaultdict(int,
                         {'MISC': 12,
                          'PERS': 22,
                          'LOC': 40,
                          'Unmatched': 105})})

In [256]:
import plotly.express as px
import pandas as pd

# Your original data
data = conf_matrix1

# Prepare lists for DataFrame construction
actual = []
predicted = []
counts = []

for (act, pred), count in data.items():
    actual.append(act)
    predicted.append('None' if pred is None else pred)  # Replace None with 'None' for better visualization
    counts.append(count)

# Create DataFrame
df = pd.DataFrame({'Actual': actual, 'Predicted': predicted, 'Count': counts})

# Pivot to format suitable for heatmap
pivot_table = df.pivot(index='Actual', columns='Predicted', values='Count').fillna(0)

# Generate heatmap
fig = px.imshow(pivot_table,
                labels=dict(x="Predicted Entity Type", y="Actual Entity Type", color="Count"),
                x=pivot_table.columns,
                y=pivot_table.index,
                text_auto=True,
                aspect="auto")

fig.update_layout(
    title="Entity Recognition Confusion Matrix",
    xaxis_title="Predicted Entity Type",
    yaxis_title="Actual Entity Type"
)

fig.show()


# Debugging

In [197]:
ENTITY = 'LOC'
entity_false_negatives = {ENTITY: Counter()}
false_negatives = set([e for e in entity_y_true if e[0] == 'LOC']) - set([e for e in entity_y_pred if e[0] == 'LOC'])
for e in false_negatives:
    t_type, t_start, t_end = e
    for pred_ent in entity_y_pred:
        p_type, p_start, p_end = pred_ent
        if t_start == p_start and t_start == p_end:
            if p_type == 'LOC':
                print(pred_ent)
            entity_false_negatives[t_type][p_type]+=1
            

ENTITY = 'LOC'
entity_false_positives = {ENTITY: Counter()}
false_positive = set([e for e in entity_y_pred if e[0] == ENTITY]) - set([e for e in entity_y_true if e[0] == ENTITY]) 
for e in false_positive:
    p_type, p_start, p_end = e
    for true_ent in entity_y_true:
        t_type, t_start, t_end = true_ent
        if t_start == p_start and t_end == p_end:
            # if p_type == 'ORG':
            #     # if t_type == 'ORG':
            #         print(true_ent)
            if p_type == t_type:
                entity_false_positives[p_type][t_type]+=1

('LOC', 16466, 16466)
('LOC', 2545, 2545)
('LOC', 1615, 1615)
('LOC', 25446, 25446)
('LOC', 8786, 8786)


In [198]:
entity_false_positives

{'LOC': Counter()}

In [201]:
id = 8786
for entity in entity_y_true:
    t, s, e = entity
    if s == id:
        print(entity)
for entity in entity_y_pred:
    t, s, e = entity
    if s == id or e == id+1:
        print(entity)

('LOC', 8786, 8787)
('LOC', 8786, 8786)
('LOC', 8787, 8787)


In [203]:
for entity in false_negatives:
    t, s, e = entity
    # if t == 'LOC':
    #     print(entity)
    if s == 8786:
        print(entity)

('LOC', 8786, 8787)


In [196]:
for entity in false_positive:
    t, s, e = entity
   
    if s == 16466 or e == 16467:
        print(entity)

('LOC', 16466, 16466)
('LOC', 16467, 16467)


In [ ]:
for entity in entity_y_true:
    t, s, e = entity
    if s == 16963:
        print(entity)

In [ ]:
for entity in entity_y_pred:
    t, s, e = entity
    if s == 16963:
        print(entity)

('LOC', 16963, 16965)


In [206]:
analysis_data.iloc[8780:8790]

,Sentence Ids,Token Positions,Words,Tokens,Word Pieces,Core Tokens,True Labels,Token Selector Id,Pred Labels,Agreements,...,Prediction Max Entropy,Token Confidence,Variability,Pre X,Pre Y,Consistency Ratio,Inconsistency Ratio,Normalized Token Entropy,Normalized Word Entropy,Normalized Prediction Entropy
8780,279,34,ما,ما,ما,ما,O,ما@#34@#279,O,True,...,3.1699,1.0000,0.3143,-2.5077,14.0135,0.997253,0.002747,0.0273,0.0273,0.000189
8781,279,35,تعتبره,تعتبره,تعتبره,تعتبره,O,تعتبره@#35@#279,O,True,...,3.1699,0.9999,0.3142,-2.1208,12.4250,0.000000,0.000000,-1.0000,-1.0000,0.000315
8782,279,36,بكين,بكين,بكين,بكين,B-LOC,بكين@#36@#279,B-LOC,True,...,3.1699,0.9832,0.3083,1.2486,8.2056,1.000000,0.000000,0.0000,0.0000,0.048614
8783,279,37,وسيول,وسي,"وسي, ##ول",وسي,B-LOC,وسي@#37@#279,B-LOC,True,...,3.1699,0.8687,0.2684,1.0763,9.0477,0.052632,0.947368,0.2975,0.2975,0.264803
8784,279,38,وسيول,##ول,"وسي, ##ول",IGNORED,IGNORED,IGNORED@#38@#279,IGNORED,True,...,3.1699,0.7048,0.2205,0.9176,9.0113,0.000000,0.000000,-1.0000,-1.0000,0.399981
8785,279,39,تمجيدا,تم,"تم, ##جيد, ##ا",تم,O,تم@#39@#279,O,True,...,3.1699,1.0000,0.3143,-2.8742,12.7942,1.000000,0.000000,0.0000,0.0000,0.000252
8786,279,40,تمجيدا,##جيد,"تم, ##جيد, ##ا",IGNORED,IGNORED,IGNORED@#40@#279,IGNORED,True,...,3.1699,0.9999,0.3142,-1.5313,11.7018,0.000000,0.000000,-1.0000,-1.0000,0.000442
8787,279,41,تمجيدا,##ا,"تم, ##جيد, ##ا",IGNORED,IGNORED,IGNORED@#41@#279,IGNORED,True,...,3.1699,1.0000,0.3143,-1.6739,11.4220,0.000000,0.000000,-1.0000,-1.0000,0.000284
8788,279,42,لليابان,لليابان,لليابان,لليابان,B-LOC,لليابان@#42@#279,B-LOC,True,...,3.1699,0.9960,0.3129,1.4297,7.9500,0.000000,0.000000,-1.0000,-1.0000,0.014417
8789,279,43,الامبريالية,الامبريالية,الامبريالية,الامبريالية,O,الامبريالية@#43@#279,O,True,...,3.1699,0.9998,0.3142,0.9539,10.7389,0.000000,0.000000,-1.0000,-1.0000,0.000852
